<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [83]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [51]:
%%sql
INSERT INTO Ticket (performance_id,username)
VALUES (6,(
    SELECT username
    FROM Customer
    WHERE (username='user2') and (user_password='h4ck3r')
    LIMIT 1
))

 * sqlite:///movies.sqlite
1 rows affected.


[]

+ Show the performance dates for one of the movies.

In [52]:
%%sql
Select *
FROM ticket

 * sqlite:///movies.sqlite
Done.


ticket_id,performance_id,username
3e5b19427b0c212bdf6fc60f1347ed23,1,user1
5dcbe186cc7de75b99c2dd0b66cd5359,1,user3
dd672e0bfc18dfa3094262ca6ba9a2cb,2,user2
3f1d2e7cacc65575d3b9e08fb6de225b,4,user3
49a1121d4b5b7e5b3a60a574e896e6a9,4,user2
b47905b2cea8f48d45b678444ec4c985,4,user1
d7f6f46cfe96f9a20e8a9a9a51c95f13,13,user1
af4d2deee90533c71f9cc605c096e213,13,user2
958d710487eb947d4d635e7719674166,1,admin
97fd87baa8c44eb81d8719f0beb031bc,1,user1


+ Show all data concerning performances at a given theatere
  on a given date.

In [5]:
%%sql
SELECT *
FROM performance
JOIN theater USING (theater_name)
WHERE theater_name='Sparta'
    and performance_date=date('2022-02-01')
    


 * sqlite:///movies.sqlite
Done.


performance_id,performance_date,start_time,IMDB_KEY,theater_name,capacity
1,2022-02-01,19:00,abcd123,Sparta,90
2,2022-02-01,19:30,abcd123,Sparta,90
3,2022-02-01,20:00,abcd123,Sparta,90
4,2022-02-01,20:30,abcd123,Sparta,90


+ List all customers

In [7]:
%%sql
SELECT username, full_name
FROM Customer


 * sqlite:///movies.sqlite
Done.


username,full_name
user1,user username
user2,user2 username2
user3,user3 username3 lastname3


+ List all tickets

In [9]:
%%sql
SELECT ticket_id
FROM ticket


 * sqlite:///movies.sqlite
Done.


ticket_id
0cbca1ddf027b402548152d3fcd412cf
16948e6d39a51de0d92ba007970906d9
48ec271991d747ab0d4de25223f80d33
50118168f738c254367c2f4a6f3d7d1a
f453b8cbf4b4c5de8f954919dad92828
f4d711faf0e6ad6cc335692e99d2f05f


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [88]:
%%sql
/* TESTER FÖR TRIGGER, KÖR ETT STYCKE I TAGET*/

/*
SELECT *
FROM Ticket
WHERE performance_id = 13
*/

/*
INSERT 
INTO Ticket (performance_id, username)
VALUES (13, 'user1'), (13, 'user2')
*/

/*
SELECT * 
FROM Ticket
WHERE performance_id = 13
*/

/*
Insert
INTO Ticket (performance_id, username)
VALUES (13, 'user3')
*/


 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) Sold out
[SQL: /* TESTER FÖR TRIGGER, KÖR ETT STYCKE I TAGET*/

/*
SELECT *
FROM Ticket
WHERE performance_id = 13
*/

/*
INSERT 
INTO Ticket (performance_id, username)
VALUES (13, 'user1'), (13, 'user2')
*/

/*
SELECT * 
FROM Ticket
WHERE performance_id = 13
*/


Insert
INTO Ticket (performance_id, username)
VALUES (13, 'user3')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [ ]:
%%sql


+ Try to insert two movie theaters with the same name (this
  should fail).

In [ ]:
%%sql


+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [ ]:
%%sql


+ Create a ticket where either the user or the performance
  doesnâ€™t exist (this should fail).

In [ ]:
%%sql
